In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Text classification

In [2]:
from datasets import load_dataset

liar = load_dataset("liar")

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
def preprocess_function(examples):
    return tokenizer(examples["statement"], truncation=True)

In [5]:
tokenized_liar = liar.map(preprocess_function, batched=True)

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

In [6]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

In [7]:
!pip install evaluate
import evaluate

accuracy = evaluate.load("accuracy")

In [8]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Train

In [9]:
label_mapping = {0: 0, 1: 0, 2: 1, 3: 1, 4: 0, 5:0}

tokenized_liar["train"] = tokenized_liar["train"].map(lambda x: {"label": label_mapping[x["label"]]})
tokenized_liar["test"] = tokenized_liar["test"].map(lambda x: {"label": label_mapping[x["label"]]})
tokenized_liar["val"] = tokenized_liar["validation"].map(lambda x: {"label": label_mapping[x["label"]]})


Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

In [30]:

id2label = {0: "FALSE", 1: "TRUE"}
label2id = {"FALSE": 0, "TRUE": 1}

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
!pip install accelerate -U
!pip install transformers[torch]

In [19]:
import torch
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_liar["train"],
    eval_dataset=tokenized_liar["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
# device = torch.device('cuda')
# model = model.to(device)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.605535,0.666407
2,0.613100,0.611493,0.668745


TrainOutput(global_step=642, training_loss=0.6081668818108389, metrics={'train_runtime': 36.0983, 'train_samples_per_second': 568.946, 'train_steps_per_second': 17.785, 'total_flos': 259586426091264.0, 'train_loss': 0.6081668818108389, 'epoch': 2.0})

In [ ]:
from transformers import AutoTokenizer

gpt2 = torch.load('gpt2gens.pt')
inputs = tokenizer.__call__(gpt2, return_tensors="pt", padding=True)
inputs.to('cuda')

with torch.no_grad():
    logits = model(**inputs).logits

for logi in logits:
  predicted_class_id = logi.argmax().item()
  print(model.config.id2label[predicted_class_id])